In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor

import numpy as np
import matplotlib.pyplot as plt
import PIL

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
torch.__version__

'2.3.0'

In [4]:
class MNISTDNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTDNN,self).__init__()
        self.fc1 = nn.Linear(IMG_SIZE*IMG_SIZE,32)
        self.BN1 = torch.nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32,10)
    
    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.fc2(x)
        x = torch.softmax(x,dim=-1)
        return x  

In [5]:
class MNISTCNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTCNN,self).__init__()
        self.conv1 = nn.Conv2d(1,8,5,stride=2)
        self.BN1 = torch.nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,8,5,stride=2)
        self.BN2 = torch.nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,8,3,stride=1)
        self.fc = nn.Linear(8*2*2,10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.BN2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = x.view(-1,8*2*2)
        x = self.fc(x)
        x = torch.softmax(x,dim=-1)
        return x

In [6]:

def compute_acc(argmax,y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i]==y[i]:
            count+=1
    return count / len(argmax)

In [7]:
IMG_SIZE = 28
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHES = 30

In [8]:
transforms = Compose([
    ToTensor(),
])

trainset = MNIST('../chapter_6_cnn/dataset/',train=True,transform=transforms,download=True)
testset = MNIST('../chapter_6_cnn/dataset/',train=False,transform=transforms,download=True)

args = {
    'num_workers' : 1,
    'batch_size' : BATCH_SIZE,
    'shuffle' : True,
}

train_loader = DataLoader(trainset,**args)
test_loader = DataLoader(testset,**args)

In [9]:
model = MNISTDNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0
    
    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()
        
    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:
        x,y = next(iter(test_loader))
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())
        
        print("Acc(val) : {}".format(test_acc))

torch.save(model.state_dict(), "../chapter_6_cnn/models/DNN.pt")

number of parameters : 25514
Epoch 1, Loss(train) : 407.9110178947449
Epoch 2, Loss(train) : 367.6132342815399
Acc(val) : 0.9609375
Epoch 3, Loss(train) : 360.90329444408417
Epoch 4, Loss(train) : 357.6088013648987
Acc(val) : 0.953125
Epoch 5, Loss(train) : 355.56522810459137
Epoch 6, Loss(train) : 354.21087980270386
Acc(val) : 0.94921875
Epoch 7, Loss(train) : 353.0325322151184
Epoch 8, Loss(train) : 352.22053492069244
Acc(val) : 0.96875
Epoch 9, Loss(train) : 351.49225652217865
Epoch 10, Loss(train) : 350.88529551029205
Acc(val) : 0.96484375
Epoch 11, Loss(train) : 350.4940279722214
Epoch 12, Loss(train) : 349.9830936193466
Acc(val) : 0.96875
Epoch 13, Loss(train) : 349.6186110973358
Epoch 14, Loss(train) : 349.2909013032913
Acc(val) : 0.984375
Epoch 15, Loss(train) : 349.0419228076935
Epoch 16, Loss(train) : 348.83533132076263
Acc(val) : 0.9609375
Epoch 17, Loss(train) : 348.5632222890854
Epoch 18, Loss(train) : 348.35739970207214
Acc(val) : 0.97265625
Epoch 19, Loss(train) : 348.16

RuntimeError: Parent directory ../chapter_6_cnn/models does not exist.

In [10]:
torch.save(model.state_dict(), "../chapter_6_cnn/models/DNN.pt")

In [36]:
trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [39]:
print(model)

MNISTDNN(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (BN1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)


In [41]:
loss_fn

CrossEntropyLoss()

In [47]:
for x, y in train_loader:
    print(y)

tensor([8, 1, 7, 8, 4, 2, 6, 8, 2, 4, 8, 7, 5, 1, 1, 3, 4, 5, 9, 9, 3, 5, 5, 6,
        9, 7, 2, 9, 5, 0, 2, 2, 0, 4, 1, 8, 7, 7, 5, 2, 8, 9, 3, 1, 6, 9, 0, 2,
        6, 9, 1, 8, 0, 5, 2, 4, 6, 9, 4, 5, 8, 8, 1, 4, 4, 4, 9, 3, 9, 2, 7, 6,
        1, 3, 2, 5, 6, 5, 0, 0, 7, 7, 2, 7, 7, 1, 3, 6, 5, 3, 1, 0, 5, 0, 1, 9,
        8, 7, 1, 3, 0, 7, 7, 2, 8, 6, 3, 1, 9, 6, 6, 0, 9, 4, 8, 6, 0, 4, 1, 8,
        4, 5, 0, 6, 3, 6, 3, 4, 4, 8, 6, 1, 1, 6, 2, 6, 4, 4, 4, 2, 1, 0, 1, 2,
        3, 0, 4, 0, 4, 1, 1, 6, 3, 8, 2, 9, 7, 2, 9, 2, 2, 4, 5, 8, 7, 0, 8, 8,
        3, 2, 1, 8, 6, 5, 5, 3, 3, 7, 3, 5, 6, 4, 7, 5, 1, 8, 5, 0, 3, 8, 0, 3,
        6, 4, 0, 9, 1, 8, 8, 5, 8, 1, 5, 3, 3, 0, 5, 7, 0, 7, 3, 8, 2, 1, 3, 8,
        8, 1, 7, 3, 9, 4, 0, 1, 0, 3, 2, 5, 9, 2, 7, 7, 6, 3, 1, 5, 7, 7, 2, 5,
        8, 4, 0, 0, 9, 8, 4, 6, 3, 4, 6, 8, 3, 9, 3, 8])
tensor([0, 8, 0, 7, 9, 5, 4, 1, 2, 7, 1, 5, 5, 6, 9, 6, 1, 5, 4, 1, 4, 9, 6, 3,
        0, 3, 5, 8, 8, 6, 0, 3, 3, 6, 1, 0, 9, 2, 4, 6, 3, 4, 2

In [48]:
loss

tensor(1.4887, device='cuda:0', grad_fn=<NllLossBackward0>)

In [49]:
y_

tensor([[1.6208e-10, 1.5124e-15, 1.0000e+00,  ..., 1.0318e-10, 6.3672e-11,
         7.9840e-10],
        [1.9651e-16, 3.8407e-15, 1.6260e-19,  ..., 8.6929e-11, 6.5466e-14,
         4.8835e-11],
        [8.6161e-09, 2.1384e-12, 4.1127e-10,  ..., 5.5734e-11, 3.5426e-08,
         4.6580e-07],
        ...,
        [1.3633e-13, 1.0000e+00, 1.4438e-11,  ..., 2.5472e-10, 2.6938e-06,
         1.3435e-08],
        [2.3329e-13, 5.6320e-16, 5.0565e-10,  ..., 8.0373e-17, 8.9455e-11,
         1.0936e-12],
        [7.0781e-13, 7.8860e-14, 1.2475e-15,  ..., 6.6871e-10, 1.5389e-10,
         8.9596e-06]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [50]:
y

tensor([9, 5, 3, 3, 4, 6, 5, 5, 6, 6, 1, 3, 3, 5, 1, 6, 7, 2, 9, 0, 3, 1, 1, 9,
        2, 2, 7, 4, 8, 5, 7, 5, 7, 1, 2, 7, 2, 6, 7, 7, 0, 5, 4, 4, 3, 8, 8, 3,
        6, 0, 8, 9, 7, 0, 9, 5, 3, 9, 1, 1, 6, 0, 7, 4, 9, 1, 4, 3, 6, 1, 9, 7,
        7, 1, 3, 6, 1, 0, 6, 5, 9, 8, 8, 9, 7, 4, 0, 1, 4, 1, 2, 6, 7, 8, 0, 4])

In [51]:
oh_y = F.one_hot(y, num_classes=y_.shape[1]).float()
oh_y

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [9]:
target = torch.tensor([[0.2, 0.5, 0.3], [0.3, 0.2, 0.5]])

In [11]:
target.argmax(dim=-1)

tensor([1, 2])

In [16]:


X = np.load('../generated_games/features-200.npy')
Y = np.load('../generated_games/labels-200.npy')

In [18]:
Y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])